参考文档：[Azure OpenAI 入门教程 - LangChain 篇 ：Prompt 提示词模板](https://zhuanlan.zhihu.com/p/642996705)

In [ ]:
! pip install langchain
! pip install openai
! pip install python-dotenv

In [ ]:
import dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from pprint import pprint

dotenv.load_dotenv()
llm = ChatOpenAI()

In [ ]:
template_string = """用英文翻译以下文本```{text}```，翻译时采用 {style} 风格"""
prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """商用英式英语"""
customer_text = """六一儿童节，可以放假半天，大家收拾好东西准备回家"""

customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_text)

customer_response = llm(customer_messages)

pprint(customer_response.content)

In [ ]:
customer_review = """\
这款吹风机有四个档位：小风，中风，大风和特大风。\
它还有2天能到货，正好赶上我妻子的周年纪年礼物。\
虽然价格略高于其他吹风机，但我认为其额外功能是值得的。
"""

review_template = """\
对于文本，提取以下信息：

gift: 该物品是否作为礼物购买？如果是，请回答True；否则或未知请回答False。

delivery_days: 产品到达需要多少天？如果找不到此信息，请输出-1.

将输出苌转化为JSON，具有以下键：
gift
delivery_days

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)

response = llm(messages)
print(response.content)

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

review_template_2 = """\
对于文本，提取以下信息：

gift: 该物品是否作为礼物购买？如果是，请回答True；否则或未知请回答False。

delivery_days: 产品到达需要多少天？如果找不到此信息，请输出-1.

text: {text}

{format_instructions}
"""

gift_schema = ResponseSchema(name="gift",
                             description="该物品是否作为礼物购买？如果是，请回答True；否则或未知请回答False。")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="产品到达需要多少天？如果找不到此信息，请输出-1.")
response_schema = [gift_schema, delivery_days_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review,
                                    format_instructions=format_instructions)
response = llm(messages)
print(response.content)
output_dict = output_parser.parse(response.content)
print(output_dict.get('delivery_days'))